In [34]:
## ARTIFICIAL NEURAL NETWORK FOR RECOGNITION OF HANDWRITTEN LETTERS ##

In [60]:
import numpy as np
from sklearn.datasets import load_digits
import random
import gzip
import pickle

In [61]:
# Load MNIST dataset
def load_data():
    f = gzip.open('./data/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1') # Needed due to python 2v3 pickle issues
    f.close()
    return (training_data, validation_data, test_data)
    
def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

In [62]:
load_data_wrapper()

(<zip at 0x115a53788>, <zip at 0x115a53c88>, <zip at 0x115a53208>)

In [63]:
def sigmoid(x):
    return 1.0/(1.0+np.exp(z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [68]:
class Network(object):
    
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y,1) for y in sizes[1:]]
        self.weights = [np.random.randn(y,x) for x,y in zip(sizes[:-1],sizes[1:])]
        
    def feedforward(self,a):
        """Returns the output of the network if 'a' is input"""
        for b,w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w,a)+b)
        return a
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """Train the neural network using mini-batch stochastic gradient descent. The 'training_data' is a list of
        tuples '(x,y)' representing the training inputs and the desired outputs. If 'test_data' is provided, the
        network will be evaluated against the test data after each eopoch and partial progress printed out for ease
        of tracking and measurement."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs): # range() returns object rather than a list of seq()
            #random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0,n,mini_batch_size)]
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if (test_data):
                print("Epoch {0}: {1}/{2}").format(j,self.evaluate(test_data),n_test)
            else:
                print("Epoch {0} complete").format(j)
    
    def update_mini_batch(self, mini_batch, eta):
        """UPdate the network's weights and biases by applying gradient descent using backpropogation to a single
        mini_batch. the 'mini_batch' is a list of tuples '(x,y)' and 'eta' is the learning rate"""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x,y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        
        self.weights = [w - (eta/len(mini_batch))*nw for w,nw in zip(self.weights,nabla_w)]
        self.biases = [b - (eta/len(mini_batch))*nb for b,nb in zip(self.biases,nabla_b)]
    
    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives (partial C_x / partial a) for the output activations."""
        return (output_activations-y)
    
        
    def backprop(self, x, y):
        """Return a tuple '(nabla_b, nabla_w)' representing the gradient for the cost function C_x.  They are the 
        layer-by-layer lists of numpy arrays, similar to self.biases and self.weights."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # Feedforward
        activation = x
        activations = [x]
        zs = []
        for b,w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        
        # Backward pass
        deslta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        # Note that l=1 => last layer of neurons, l=2 => second to last layer
        
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose, delta) *sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    def evaluate(self, test_data):
        """Return number of correctly predicted digits"""
        test_results = [(np.argmax(self.feedforward(x)), y) for (x,y) in test_data]
        return sum(int(x==y) for (x,y) in test_results)

In [69]:
net = Network([784,30,10])

In [70]:
net.SGD(training_data, 30, 10, 3.0)

ValueError: too many values to unpack (expected 2)